In [1]:
import scanpy as sc
data_dir = '../../dataset/SpaGE_imputed_Xenium_breast_cancer_sample1_replicate1.h5ad'
adata = sc.read_h5ad(data_dir)

In [2]:
adata.var.index

Index(['ABCC11', 'ACTA2', 'ACTG2', 'ADAM9', 'ADGRE5', 'ADH1B', 'ADIPOQ',
       'AGR3', 'AHSP', 'AIF1',
       ...
       'ZW10', 'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B',
       'ZYX', 'ZZEF1'],
      dtype='object', length=18085)

In [3]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
from torch.optim import AdamW
import lightning.pytorch as pl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from self_supervision.models.lightning_modules.cellnet_autoencoder import MLPBarlowTwins
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

# 设置你的 .ckpt 文件路径
ckpt_path = "../../sc_pretrained/Pretrained Models/BarlowTwins.ckpt"

# 模型参数
units_encoder = [512, 512, 256, 256, 64]

# 初始化 EstimatorAutoEncoder 实例
estim = EstimatorAutoEncoder(data_path=None)  # 如果没有实际数据路径，可以设置为None

# 加载预训练模型
estim.model = MLPBarlowTwins(
        gene_dim=19331,  # 根据你的数据调整
        batch_size=128,  # 根据你的需要调整
        units_encoder=units_encoder,
        CHECKPOINT_PATH=ckpt_path
    )


estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
estim.model

/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/home/hanchuangyi/miniconda3/envs/ssl/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")
GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


MLPBarlowTwins(
  (train_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=train_
  )
  (val_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=val_
  )
  (test_metrics): MetricCollection(
    (explained_var_uniform): ExplainedVariance()
    (explained_var_weighted): ExplainedVariance()
    (mse): MeanSquaredError(),
    prefix=test_
  )
  (inner_model): MLP(
    (0): Linear(in_features=19331, out_features=512, bias=True)
    (1): SELU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): SELU()
    (5): Dropout(p=0.0, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): SELU()
    (8): Dropout(p=0.0, inplace=False)
    (9): Linear(in_features=256, out_features=25

In [4]:
# 加载 checkpoint
checkpoint = torch.load(ckpt_path)
estim.model.inner_model.load_state_dict({k.replace('backbone.', ''): v for k, v in checkpoint.items() if 'backbone' in k})

# 5. 添加分类层 (FC 层)
n_classes = len(adata.obs['cell_type'].unique())
estim.model.fc = nn.Linear(units_encoder[-1], n_classes)

In [5]:
# 6. 冻结 encoder 的大部分参数，解冻所有层
for param in estim.model.inner_model.parameters():
    param.requires_grad = False  # 冻结所有层

# 解冻最后两层
for param in list(estim.model.inner_model.parameters())[-5:]:
    param.requires_grad = True

In [6]:
# 7. 定义损失函数和优化器
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
estim.model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(filter(lambda p: p.requires_grad, estim.model.parameters()), lr=9e-4, weight_decay=0.05)

In [7]:
import pandas as pd
import numpy as np

var_df = pd.read_parquet('../../sc_pretrained/var.parquet')
all_genes = var_df['feature_name'].tolist()

new_data = np.zeros((adata.X.shape[0], len(all_genes)), dtype=np.float32)

adata.var['gene_name']=adata.var.index
existing_genes = adata.var['gene_name']

In [8]:
# 将所有基因名称转换为小写
all_genes_lower = [gene.lower() for gene in all_genes]
adata_genes_lower = [gene.lower() for gene in existing_genes]

# 将两个列表转换为集合
all_genes_set = set(all_genes_lower)
adata_genes_set = set(adata_genes_lower)

# 计算交集
matching_genes = all_genes_set.intersection(adata_genes_set)
matching_count = len(matching_genes)
# 计算不匹配的基因
non_matching_genes = adata_genes_set - matching_genes
non_matching_count = len(non_matching_genes)


# 输出结果
print(f"匹配的基因数量: {matching_count}")
print(f"匹配的基因列表: {matching_genes}")
non_matching_genes

匹配的基因数量: 17773
匹配的基因列表: {'il22', 'pramef27', 'clec4g', 'mob2', 'c19orf53', 'hsd3b1', 'foxc2', 'dbnl', 'agfg2', 'anxa8', 'cntnap4', 'smpx', 'ppef2', 'kifap3', 'lrrc58', 'gbp7', 'mrgprx4', 'bmp15', 'lrrc73', 'szt2', 'lrrk2', 'nlrp14', 'ier5', 'kirrel3', 'c20orf202', 'deaf1', 'iqcb1', 'iqcn', 'klrb1', 'blnk', 'tvp23a', 'eif3m', 'znf107', 'cpsf4', 'plekhg4b', 'ice2', 'rimbp3b', 'exoc6', 'fam114a2', 'tmem144', 'uck1', 'znf41', 'r3hdm1', 'chst4', 'siah3', 'wdr45b', 'dguok', 'trmt5', 'lix1l', 'otof', 'fbxl17', 'rxfp2', 'zdhhc3', 'col5a3', 'f2rl1', 'clec2l', 'tmem14c', 'ptf1a', 'cdca5', 'nmnat3', 'spag1', 'rbmxl1', 'fgfbp2', 'fgl2', 'chst5', 'pmpca', 'kiaa1217', 'ncbp2', 'trim8', 'mesd', 'tmem160', 'yjefn3', 'gmfb', 'rab36', 'mmp15', 'ano8', 'bpifb1', 'myh10', 'inpp4a', 'krtdap', 'nr3c2', 'llph', 'cd79b', 'polr2h', 'ccnq', 'cd226', 'safb2', 'xpr1', 'clba1', 'hmhb1', 'insig1', 'vash1', 'hdac6', 'arrdc5', 'prg4', 'cyp26c1', 'taf7', 'cltrn', 'trmt10a', 'gdi2', 'ngrn', 'oosp1', 'tmem145', 'prrx2',

{'aars',
 'ac004593.3',
 'ac005551.1',
 'ac007244.1',
 'ac007906.2',
 'ac008397.1',
 'ac008687.4',
 'ac010255.3',
 'ac010325.1',
 'ac011005.1',
 'ac011195.2',
 'ac013470.2',
 'ac015871.1',
 'ac021072.1',
 'ac021097.2',
 'ac025283.2',
 'ac025287.4',
 'ac067752.1',
 'ac072022.2',
 'ac073111.4',
 'ac087498.1',
 'ac090360.1',
 'ac091057.6',
 'ac092835.1',
 'ac093323.1',
 'ac099489.1',
 'ac104389.5',
 'ac106774.4',
 'ac113348.1',
 'ac115220.1',
 'ac118549.1',
 'ac119396.1',
 'ac132217.2',
 'ac134684.8',
 'ac135068.1',
 'ac138647.1',
 'ac187653.1',
 'ac233723.1',
 'ac236972.4',
 'acpp',
 'adprhl2',
 'adss',
 'adssl1',
 'al032819.3',
 'al109810.2',
 'al121578.2',
 'al135905.2',
 'al160269.1',
 'al162231.1',
 'al162596.1',
 'al353572.3',
 'al354761.1',
 'al391650.1',
 'al445238.1',
 'al451007.3',
 'al590560.2',
 'al603764.2',
 'al772284.2',
 'al845331.2',
 'alg1l',
 'ap000552.4',
 'ap002495.1',
 'arih2os',
 'armc4',
 'arse',
 'atp5md',
 'atp5mpl',
 'atp6ap1l',
 'b3gnt10',
 'bhmg1',
 'bx255925.

In [9]:
gene_to_index = {gene: idx for idx, gene in enumerate(all_genes_lower)}
dense_adata_X = adata.X
for i, gene in enumerate(adata_genes_lower):
    if gene in gene_to_index:
        new_data[:, gene_to_index[gene]] = dense_adata_X[:, i]
    else:
        print(f'Gene {gene} not found in all_genes list')

Gene fam49a not found in all_genes list
Gene kars not found in all_genes list
Gene lars not found in all_genes list


Gene nars not found in all_genes list
Gene polr2j3 not found in all_genes list
Gene qars not found in all_genes list


Gene trac not found in all_genes list
Gene wars not found in all_genes list
Gene aars not found in all_genes list


Gene ac004593.3 not found in all_genes list
Gene ac005551.1 not found in all_genes list
Gene ac007244.1 not found in all_genes list
Gene ac007906.2 not found in all_genes list
Gene ac008397.1 not found in all_genes list
Gene ac008687.4 not found in all_genes list
Gene ac010255.3 not found in all_genes list
Gene ac010325.1 not found in all_genes list
Gene ac011005.1 not found in all_genes list
Gene ac011195.2 not found in all_genes list
Gene ac013470.2 not found in all_genes list
Gene ac015871.1 not found in all_genes list
Gene ac021072.1 not found in all_genes list
Gene ac021097.2 not found in all_genes list
Gene ac025283.2 not found in all_genes list
Gene ac025287.4 not found in all_genes list
Gene ac067752.1 not found in all_genes list
Gene ac072022.2 not found in all_genes list
Gene ac073111.4 not found in all_genes list
Gene ac087498.1 not found in all_genes list
Gene ac090360.1 not found in all_genes list
Gene ac091057.6 not found in all_genes list
Gene ac092835.1 not found in all

Gene acpp not found in all_genes list


...

Gene twistnb not found in all_genes list


Gene vars not found in all_genes list


Gene wdr34 not found in all_genes list
Gene wdr60 not found in all_genes list
Gene wdr63 not found in all_genes list
Gene wdr66 not found in all_genes list
Gene wdr78 not found in all_genes list
Gene wdyhv1 not found in all_genes list


Gene yars not found in all_genes list
Gene z82206.1 not found in all_genes list


Gene znf883 not found in all_genes list
Gene znrd1 not found in all_genes list


In [10]:
estim.trainer = pl.Trainer(accelerator="gpu", devices=1 if torch.cuda.is_available() else None)
# 5. 数据集划分（70% 训练，15% 验证，15% 测试）
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(adata.obs['cell_type'])  # 预先编码标签


random_seed = 42

X_train_val, X_test, y_train_val, y_test = train_test_split(
    new_data, labels_encoded, test_size=0.15, random_state=random_seed)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1765, random_state=random_seed)  # 0.1765 是为了让验证集占 15%

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs


In [11]:
X_train_tensor = torch.tensor(X_train).float().to(device)
y_train_tensor = torch.tensor(y_train).long().to(device)
X_val_tensor = torch.tensor(X_val).float().to(device)
y_val_tensor = torch.tensor(y_val).long().to(device)

In [12]:
from torch.utils.data import DataLoader, TensorDataset

# 设置 batch size
batch_size = 128  # 根据实际需求调整 batch size

# 使用 TensorDataset 将训练数据和标签打包
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

# 使用 DataLoader 来创建批次
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# 7. 微调模型
def train_epoch(model, optimizer, loss_fn, train_loader, val_loader):
    model.train()
    total_train_loss = 0
    
    # 训练集批次训练
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model.inner_model(X_batch)
        logits = model.fc(outputs)
        
        # 计算损失
        loss = loss_fn(logits, y_batch)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
    
    # 验证集
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            val_outputs = model.inner_model(X_val_batch)
            val_logits = model.fc(val_outputs)
            val_loss = loss_fn(val_logits, y_val_batch)
            total_val_loss += val_loss.item()

    # 返回平均损失
    return total_train_loss / len(train_loader), total_val_loss / len(val_loader)

In [13]:
torch.cuda.empty_cache()

In [14]:
import copy  # 用于保存模型的最佳状态

# Early Stopping 参数
patience = 20  # 如果验证损失在 10 个 epoch 中没有改善，停止训练
min_delta = 1e-4  # 最小改善幅度
patience_counter = 0
best_val_loss = float('inf')  # 初始设置为正无穷大
best_model_weights = copy.deepcopy(estim.model.state_dict())  # 保存最佳模型权重
train_losses = []
val_losses = []

# 训练 500 个 epoch
for epoch in range(500):
    train_loss, val_loss = train_epoch(estim.model, optimizer, loss_fn, train_loader, val_loader)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss}, Validation Loss: {val_loss}')
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    # Early Stopping 检查
    if val_loss < best_val_loss - min_delta:
        best_val_loss = val_loss  # 更新最佳验证损失
        patience_counter = 0  # 重置 patience 计数器
        best_model_weights = copy.deepcopy(estim.model.state_dict())  # 保存当前最佳模型
        print(f"Validation loss improved to {best_val_loss}, resetting patience.")
    else:
        patience_counter += 1
        print(f"No improvement in validation loss. Patience counter: {patience_counter}/{patience}")
    
    # 如果 patience_counter 超过设置的 patience，停止训练
    if patience_counter >= patience:
        print(f"Early stopping triggered. Stopping training at epoch {epoch+1}.")
        break

# 恢复到训练中性能最好的模型权重
estim.model.load_state_dict(best_model_weights)
print("Loaded best model weights based on validation loss.")

Epoch 1, Train Loss: 0.7374684349968542, Validation Loss: 0.6420267352900529
Validation loss improved to 0.6420267352900529, resetting patience.


Epoch 2, Train Loss: 0.6317355510391182, Validation Loss: 0.5902273278248492
Validation loss improved to 0.5902273278248492, resetting patience.


Epoch 3, Train Loss: 0.5949614981031106, Validation Loss: 0.5792692480656096
Validation loss improved to 0.5792692480656096, resetting patience.


Epoch 4, Train Loss: 0.5718579281259466, Validation Loss: 0.5471968026028067
Validation loss improved to 0.5471968026028067, resetting patience.


Epoch 5, Train Loss: 0.555242520077296, Validation Loss: 0.5314212406649808
Validation loss improved to 0.5314212406649808, resetting patience.


...

Epoch 82, Train Loss: 0.4400893343624726, Validation Loss: 0.44163986072322436
No improvement in validation loss. Patience counter: 16/20


Epoch 83, Train Loss: 0.4426910626602588, Validation Loss: 0.4888936712354573
No improvement in validation loss. Patience counter: 17/20


Epoch 84, Train Loss: 0.4402769491530971, Validation Loss: 0.4196418248457352
No improvement in validation loss. Patience counter: 18/20


Epoch 85, Train Loss: 0.44019047964632124, Validation Loss: 0.42110659734246697
No improvement in validation loss. Patience counter: 19/20


Epoch 86, Train Loss: 0.4394744101562791, Validation Loss: 0.42543730654087164
No improvement in validation loss. Patience counter: 20/20
Early stopping triggered. Stopping training at epoch 86.
Loaded best model weights based on validation loss.


In [15]:
# 8. 使用 KNN 替代测试阶段的 FC 分类层
# 使用 encoder 提取训练集和测试集的 embedding
estim.model.eval()
with torch.no_grad():
    train_embeddings = estim.model.inner_model(torch.tensor(X_train).float().to(device)).cpu().numpy()
    test_embeddings = estim.model.inner_model(torch.tensor(X_test).float().to(device)).cpu().numpy()

In [16]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

    

    # 初始化和训练KNN分类器
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_embeddings, y_train)
    
    # 模型预测
predictions = knn.predict(test_embeddings)

    # 计算准确率和 F1 分数
accuracy = accuracy_score(y_test, predictions)
print(f"KNN Accuracy on Test Data: {accuracy}")
f1 = f1_score(y_test, predictions, average='weighted')
print(f"Weighted F1 Score: {f1}")
    
macro_f1 = f1_score(y_test, predictions, average='macro')
print(f'Macro F1 Score: {macro_f1}')

    # 计算随机猜测的准确率
class_probabilities = np.bincount(y_test) / len(y_test)
random_accuracy = np.sum(class_probabilities ** 2)
print(f"Random Guess Accuracy: {random_accuracy}")

    # 生成分类报告
report = classification_report(y_test, predictions, target_names=label_encoder.classes_)
print(report)

KNN Accuracy on Test Data: 0.8451146342432551
Weighted F1 Score: 0.841425129962485
Macro F1 Score: 0.7202468521628832
Random Guess Accuracy: 0.1320247177116093
                         precision    recall  f1-score   support

                B_Cells       0.84      0.88      0.86       772
           CD4+_T_Cells       0.73      0.80      0.77      1286
           CD8+_T_Cells       0.75      0.73      0.74      1026
                 DCIS_1       0.82      0.87      0.85      1937
                 DCIS_2       0.76      0.71      0.74      1746
            Endothelial       0.90      0.93      0.91      1348
              IRF7+_DCs       0.86      0.76      0.81        74
         Invasive_Tumor       0.89      0.91      0.90      5230
             LAMP3+_DCs       0.79      0.61      0.69        49
          Macrophages_1       0.84      0.87      0.85      1692
          Macrophages_2       0.68      0.60      0.63       223
             Mast_Cells       0.73      0.39      0.51     

In [17]:
with torch.no_grad():
    new_data_tensor = torch.tensor(new_data).float().to(device)
    SSL_embeddings = estim.model.inner_model(new_data_tensor).detach().cpu().numpy()
new_adata = sc.read_h5ad(data_dir)
new_adata.obsm[f'SSL_BT_FT_{random_seed}'] = SSL_embeddings
new_adata.uns[f'BT_FT_y_test_{random_seed}'] = y_test
new_adata.uns[f'BT_FT_predictions_{random_seed}'] = predictions
new_adata.uns[f'BT_FT_target_names_{random_seed}'] = label_encoder.classes_
new_adata.uns[f'BT_FT_train_loss_{random_seed}'] = train_losses
new_adata.uns[f'BT_FT_val_loss_{random_seed}'] = val_losses
new_adata.write_h5ad(data_dir)

In [18]:

import pandas as pd
import os
import re

# 当前 Notebook 文件名
notebook_name = "SpaGE_imputed_Xenium_breast_cancer_sample1_replicate1_barlow_twins_fine_tune_42.ipynb"

# 初始化需要打印的值
init_train_loss = train_losses[0] if 'train_losses' in globals() else None
init_val_loss = val_losses[0] if 'val_losses' in globals() else None
converged_epoch = len(train_losses) - patience if 'train_losses' in globals() else None
converged_val_loss = best_val_loss if 'best_val_loss' in globals() else None

# 打印所有所需的指标
print("Metrics Summary:")
if 'train_losses' in globals():
    print(f"init_train_loss\tinit_val_loss\tconverged_epoch\tconverged_val_loss\tmacro_f1\tweighted_f1\tmicro_f1")
    print(f"{init_train_loss:.3f}\t{init_val_loss:.3f}\t{converged_epoch}\t{converged_val_loss:.3f}\t{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")
else:
    print(f"macro_f1\tweighted_f1\tmicro_f1")
    print(f"{macro_f1:.3f}\t{f1:.3f}\t{accuracy:.3f}")

# 保存结果到 CSV 文件
output_data = {
    'dataset_split_random_seed': [int(random_seed)],
    'dataset': ['SpaGE_imputed_xenium_breast_cancer_sample1_replicate1'],
    'method': [re.search(r'replicate1_(.*?)_\d+', notebook_name).group(1)],
    'init_train_loss': [init_train_loss if init_train_loss is not None else ''],
    'init_val_loss': [init_val_loss if init_val_loss is not None else ''],
    'converged_epoch': [converged_epoch if converged_epoch is not None else ''],
    'converged_val_loss': [converged_val_loss if converged_val_loss is not None else ''],
    'macro_f1': [macro_f1],
    'weighted_f1': [f1],
    'micro_f1': [accuracy]
}
output_df = pd.DataFrame(output_data)

# 保存到当前目录下名为 results 的文件夹中
if not os.path.exists('results'):
    os.makedirs('results')

csv_filename = f"results/{os.path.splitext(notebook_name)[0]}_results.csv"
output_df.to_csv(csv_filename, index=False)


Metrics Summary:
init_train_loss	init_val_loss	converged_epoch	converged_val_loss	macro_f1	weighted_f1	micro_f1
0.737	0.642	66	0.412	0.720	0.841	0.845
